In [1]:
# Model Building and Evaluation for Solar Panel Detection
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import cv2
from ultralytics import YOLO
import supervision as sv
from supervision.metrics.detection import MeanAveragePrecision
from supervision.metrics.detection import ConfusionMatrix
from tqdm.auto import tqdm
import pandas as pd
import yaml
import seaborn as sns
from sklearn.model_selection import train_test_split
from pathlib import Path
import rasterio
import shutil

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Create base directory for the project
base_dir = os.path.abspath("solar_panel_project")
os.makedirs(base_dir, exist_ok=True)

In [2]:
# Data Preparation and Splitting
labels_dir = "labels_native"
labels_hd_dir = "labels_hd"  # Not used in this script but kept for reference
images_dir = "image_chips_native"

# Get all image and label files
image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.tif')])
label_files = sorted([f for f in os.listdir(labels_dir) if f.endswith('.txt')])

print(f"Total images: {len(image_files)}")
print(f"Total label files: {len(label_files)}")

# Create matched pairs of images and labels
matched_files = []
for img_file in image_files:
    base_name = os.path.splitext(img_file)[0]
    label_file = base_name + '.txt'
    if label_file in label_files:
        matched_files.append((img_file, label_file))

print(f"Matched image-label pairs: {len(matched_files)}")

# Split data into train, validation, and test sets
train_val_files, test_files = train_test_split(matched_files, test_size=0.2, random_state=42)
train_files, val_files = train_test_split(train_val_files, test_size=0.125, random_state=42)  # 10% of original data

print(f"Train files: {len(train_files)}")
print(f"Validation files: {len(val_files)}")
print(f"Test files: {len(test_files)}")

# Create directories for the dataset splits
os.makedirs(os.path.join(base_dir, 'dataset/images/train'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'dataset/images/val'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'dataset/images/test'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'dataset/labels/train'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'dataset/labels/val'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'dataset/labels/test'), exist_ok=True)

# Copy files to their respective directories (using shutil instead of os.system)
def copy_files(file_pairs, split_type):
    for img_file, label_file in file_pairs:
        # Copy image
        img_src = os.path.join(images_dir, img_file)
        img_dst = os.path.join(base_dir, f'dataset/images/{split_type}/{img_file}')
        shutil.copy2(img_src, img_dst)
        
        # Copy label
        lbl_src = os.path.join(labels_dir, label_file)
        lbl_dst = os.path.join(base_dir, f'dataset/labels/{split_type}/{label_file}')
        shutil.copy2(lbl_src, lbl_dst)

copy_files(train_files, 'train')
copy_files(val_files, 'val')
copy_files(test_files, 'test')

# Create dataset.yaml file for YOLO training
dataset_config = {
    'path': os.path.join(base_dir, 'dataset'),
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'names': {0: 'solar_panel'}  # Single class - solar panels
}

with open(os.path.join(base_dir, 'dataset.yaml'), 'w') as f:
    yaml.dump(dataset_config, f, default_flow_style=False)

Total images: 2553
Total label files: 2542
Matched image-label pairs: 2542
Train files: 1778
Validation files: 255
Test files: 509


In [4]:
# Train YOLO Model
# Initialize YOLO model - you can choose different model sizes based on your needs
model = YOLO('yolov8n.pt')  # Using nano model, can also use 'yolov8s.pt', 'yolov8m.pt', etc.

# Train the model
results = model.train(
    data=os.path.join(base_dir, 'dataset.yaml'),
    epochs=25,
    imgsz=416,
    patience=10,  # Early stopping
    batch=16,
    device='cpu',
    save=True,
    name='solar_panel_detection'
)

Ultralytics 8.3.86  Python-3.11.7 torch-2.4.1+cpu CPU (12th Gen Intel Core(TM) i5-1240P)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset.yaml, epochs=25, time=None, patience=10, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=solar_panel_detection3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=

train: Scanning C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\labels\train.cache

train: WARNING  C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\images\train\solarpanels_native_1__x0_0_y0_6963_dxdy_416.tif: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING  C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\images\train\solarpanels_native_1__x0_0_y0_7229_dxdy_416.tif: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING  C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\images\train\solarpanels_native_1__x0_0_y0_7249_dxdy_416.tif: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING  C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\images\train\solarpanels_native_1__x0_1300_y0_10050_dxdy_416.tif: ignoring corrupt image/label:


val: Scanning C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\labels\val.cache... 

val: WARNING  C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\images\val\solarpanels_native_1__x0_0_y0_7119_dxdy_416.tif: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING  C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\images\val\solarpanels_native_1__x0_0_y0_7230_dxdy_416.tif: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING  C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\images\val\solarpanels_native_1__x0_128_y0_7056_dxdy_416.tif: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING  C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\images\val\solarpanels_native_1__x0_1501_y0_10325_dxdy_416.tif: ignoring corrupt image/label: Label class 2

...

      25/25         0G     0.8802      0.522     0.9092        138        416: 100%|██████████| 101/101 [08:17<00:00,  
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:24<0

                   all        226       2738       0.94      0.893      0.955      0.735



25 epochs completed in 9.399 hours.
Optimizer stripped from runs\detect\solar_panel_detection3\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\solar_panel_detection3\weights\best.pt, 6.2MB

Validating runs\detect\solar_panel_detection3\weights\best.pt...
Ultralytics 8.3.86  Python-3.11.7 torch-2.4.1+cpu CPU (12th Gen Intel Core(TM) i5-1240P)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:22<0


                   all        226       2738      0.939      0.893      0.954      0.735
Speed: 2.3ms preprocess, 67.7ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs\detect\solar_panel_detection3
